# TSP with Subtour Elimination

Given a set of cities and distance between every pair of cities, the problem is to find the shortest possible route that visits every city exactly once and returns back to the starting point.

This problem is from Chapter 3, Section 1 (3.1 Prototype Example) of the Introduction to Operations Research,
7th edition by Hillier and Lieberman.

<div>
<img src="img_9.3-1_table.png" width="400"/>
</div>

<!-- ![title](img_9.3-1_network.png) -->
<div>
<img src="img_9.3-1_network.png" width="700"/>
</div>

reference:
https://ozgurakgun.github.io/ModRef2017/files/ModRef2017_MTSP.pdf

<div class="alert alert-block alert-warning">
<b>Branch and Cut:</b> We translate the TSP problem into an assignment problem (AP) and then we add constraints to remove the subtours.<br><br>
     
We solve as an AP becuase the AP has a **unimodular constraint matrix** which results in solving for integer solutions quickly. <br><br>
    
For example, we could use the **Hungarian algorithm** which gaunreentees an integer solution<br>
<br>
    
**11:33 Lec 19**
The assignment problem has this nice structure such that you can solve it as a LP and you get an integer solution for free. So I can take advantage of that and I can solve the AP with another algorithm that is even better than simplex. So there is a Hungarian method. So you could envision doing this branch and bound without even using a solver. You are just using, just drawing the tree and keeping track of the nodes and keeping track of the subproblems with another algorithm. You don't always need to use simplex. 
    

</div>

## Import packages

In [1]:
import pandas as pd
from docplex.mp.model import Model

## Intialize the data

In [2]:
df = pd.read_csv('data_cities_numbered.csv')
edges = list((t.origin, t.destination) for t in df.itertuples())
cities = set(df['destination'])
distance = dict([((t.origin, t.destination),t.distance ) for t in df.itertuples()])

## Create the model

In [3]:
m=Model('TSP_subtour_elimination')

### <font color=green>  Decision Variables

<div class="alert alert-block alert-warning">
<b>Decision Variables:</b> $x_{ij}$ and $u_{c}$

Let $x_{ij}$ = 1 if edge ${ij}$ is used, 0 otherwise.<br>
Let $u_{c}$ = the number of cities visited from the origin up to city $c$<br>
    
<br>
$x_{ij}\in (0,1)$<br>
$u_{c}\in {\rm I\!R}$
</div>

In [4]:
x = m.binary_var_dict(edges, name = 'x')
u = m.continuous_var_dict(cities, name ='u')

### <font color=green>  Objective Function

<div class="alert alert-block alert-warning">
<b>Objective Function:</b> Minimize Z
    
 $$Z = \sum_{ij } x_{ij}d_{ij}$$
</div>

In [5]:
m.minimize(m.sum(distance[e]*x[e] for e in edges))

### <font color=green>  Constraints

<div class="alert alert-block alert-warning">
<b>Constraint 1:</b> You must enter each city once
$$
\sum_{i,j \: \epsilon \:E} x_{i} = 1 
\qquad \forall  \enspace i \: \epsilon \: Cities
$$

</div>

In [6]:
for c in cities:
    m.add_constraint(m.sum(x[(i,j)] for i,j in edges if i==c)==1, ctname='city_out_%d'%c)



<div class="alert alert-block alert-warning">
<b>Constraint 2:</b> You must exit each city once
$$
\sum_{j \: \epsilon \:E} x_{j} = 1 
\qquad \forall  \enspace j \: \epsilon \: Cities
$$ 

</div>

In [7]:
for c in cities:
    m.add_constraint(m.sum(x[(i,j)] for i,j in edges if j==c)==1, ctname='city_in_%d'%c)


<div class="alert alert-block alert-info">
<b>Note:</b> In the two constraints above we explicitly prevent the model from considering $c_{ij}$ from a city to itself (which would have a distance of 0). Alternatively, let the model go through all possible $c_{ij}$ combinations but prevent it from selecting a route in which we travel from the city to itself by assigning a large value, $M$, in these cases instead of 0
    
In this case, set M = max $c_{ij}$ + 1
</div>


<div class="alert alert-block alert-warning">
<b>Subtour Elimination Constraint:</b> A subtour is a solution set in which the number of cities in that tour is less than the total number of cities in the whole problem.
    
**Lec 19 (0:12:45)**
When we branch, we want to try and break up the smaller subtour becuase it will have fewer branches. In general this will result in solving fewer number of problems (with the exception of special cases)
</div>

In [8]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: TSP_subtour_elimination

Minimize
 obj: 304 x_1_1 + 132 x_2_1 + 217 x_3_1 + 164 x_4_1 + 58 x_5_1 + 132 x_1_2
      + 304 x_2_2 + 290 x_3_2 + 201 x_4_2 + 79 x_5_2 + 217 x_1_3 + 290 x_2_3
      + 304 x_3_3 + 113 x_4_3 + 303 x_5_3 + 164 x_1_4 + 201 x_2_4 + 113 x_3_4
      + 304 x_4_4 + 196 x_5_4 + 58 x_1_5 + 79 x_2_5 + 303 x_3_5 + 196 x_4_5
      + 304 x_5_5
Subject To
 city_out_1: x_1_1 + x_1_2 + x_1_3 + x_1_4 + x_1_5 = 1
 city_out_2: x_2_1 + x_2_2 + x_2_3 + x_2_4 + x_2_5 = 1
 city_out_3: x_3_1 + x_3_2 + x_3_3 + x_3_4 + x_3_5 = 1
 city_out_4: x_4_1 + x_4_2 + x_4_3 + x_4_4 + x_4_5 = 1
 city_out_5: x_5_1 + x_5_2 + x_5_3 + x_5_4 + x_5_5 = 1
 city_in_1: x_1_1 + x_2_1 + x_3_1 + x_4_1 + x_5_1 = 1
 city_in_2: x_1_2 + x_2_2 + x_3_2 + x_4_2 + x_5_2 = 1
 city_in_3: x_1_3 + x_2_3 + x_3_3 + x_4_3 + x_5_3 = 1
 city_in_4: x_1_4 + x_2_4 + x_3_4 + x_4_4 + x_5_4 = 1
 city_in_5: x_1_5 + x_2_5 + x_3_5 + x_4_5 + x_5_5 = 1

Bounds

In [9]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

solution = m.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Found incumbent of value 1520.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 5 columns.
Reduced MIP has 10 rows, 25 columns, and 50 nonzeros.
Reduced MIP has 25 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 25 columns, and 50 nonzeros.
Reduced MIP has 25 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP se

In [10]:
m.get_solve_status()

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [11]:
solution.display()

solution for: TSP_subtour_elimination
objective: 495
x_2_1 = 1
x_5_2 = 1
x_4_3 = 1
x_3_4 = 1
x_1_5 = 1


In [12]:
lst = []
for i in x:
    if solution.get_var_value(x[i]) > 0:
        soln = (i[0], i[1], solution.get_var_value(x[i]))
        lst.append(soln)
df = pd.DataFrame.from_records(lst, columns=['starting city', 'destination city', 'solution'])
df

,starting city,destination city,solution
0,2,1,1.0
1,5,2,1.0
2,4,3,1.0
3,3,4,1.0
4,1,5,1.0


In [78]:
def subtour_checker2(lst):
    temp_lst = lst.copy()
    start = temp_lst[0][0]
    next_city = temp_lst[0][1]
    lst2 = []

    lst2.append(temp_lst.pop(0))

    for n in range(len(temp_lst)):
        n = 0
        for x in temp_lst:
            if x[0] == next_city:
                next_city  = x[1]
                lst2.append(temp_lst.pop(n))
            n = n+1
    lst3 =[]
    for i in range(len(temp_lst)):
        lst3.append(temp_lst[i][0])
    lst3.append(temp_lst[-1][1])

    lst4 = []
    for i in range(len(lst2)):
        lst4.append(lst2[i][0])  
    lst4.append(lst2[-1][1])
    
    if len(lst2) == len(temp_lst):
        print("Woohoo! No Subtours!")
        return(True)
    else:
        print("Subtours exists")
        print('*****************')
#         print('temp_lst:', temp_lst)
        print('subtour 1:', lst3)
#         print('*****************')
#         print('lst2:', lst2)
        print('subtour 2:', lst4)

        return(False)


In [80]:
subtour_checker2(lst)

Subtours exists
*****************
subtour 1: [4, 3, 4]
subtour 2: [2, 1, 5, 2]


False

In [84]:
lst2

[(2, 1, 1.0), (1, 5, 1.0), (5, 2, 1.0)]

In [100]:
var_lst = []
for i in range(len(lst2)):
    var_lst.append('branch_%d'%i)

In [101]:
var_lst

['branch_0', 'branch_1', 'branch_2']

In [102]:
temp_lst

[(2, 1, 1.0), (5, 2, 1.0), (4, 3, 1.0), (3, 4, 1.0), (1, 5, 1.0)]

In [103]:
temp_lstb = [(2, 1, 1.0), (1, 5, 1.0), (5, 2, 1.0)]

In [106]:
for i in range(len(temp_lstb)):
    var_lst[i] =(temp_lstb[i][0],temp_lstb[i][1])


In [107]:
var_lst

[(2, 1), (1, 5), (5, 2)]